This workbook implements Context Based Filtering for a Dogs Recommendation System, working from the raw data all the way to model creation and initial results.

# Table of Contents

* [Load in Data and Segment Features from Context data](#segment)
* [Pre-process feature data](#pre-process)
    - [Make Needed Helper Functions and Imports](#pp_pipeline)
* [Collaborative Filtering - Under Construction WIP](#cf)    
* [Conclusion and Next Steps](#conclusion)

# Load in Data and Segment Features from Context data<a id='segment'></a>

First, we load all of our adoptable dogs.

In [1]:
from google.colab import drive
import joblib #so I can save files out

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
dogs_DF = pd.read_csv("/content/drive/MyDrive/MLE10PetMatch/Adoptable_dogs_20221202_withExtras.csv",header=0,index_col=0)
dogs_DF.shape

(97694, 70)

In [3]:
dogs_DF = dogs_DF.drop_duplicates()
dogs_DF.shape

(97459, 70)

In [4]:
pd.set_option('display.max_columns', 500)
dogs_DF.sample(3)

,id,organization_id,url,type,species,age,gender,size,coat,tags,name,description_x,organization_animal_id,photos,videos,status,status_changed_at,published_at,distance,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,primary_photo_cropped.small,primary_photo_cropped.medium,primary_photo_cropped.large,primary_photo_cropped.full,contact.email,contact.phone,contact.address.address1,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,animal_id,animal_type,organization_id.1,primary_photo_cropped,description_y,temperament,popularity,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,group,grooming_frequency_value,grooming_frequency_category,shedding_value,shedding_category,energy_level_value,energy_level_category,trainability_value,trainability_category,demeanor_value,demeanor_category
59009,58648075,MS144,https://www.petfinder.com/dog/star-star-blake-...,Dog,Dog,Adult,Male,Large,NaN,[],**BLAKE**,"Please go to our website,\nhttp://www.hooversh...",18662245,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,[],adoptable,2022-10-25T05:41:38+0000,2022-10-25T05:41:38+0000,NaN,Labrador Retriever,Spaniel,True,False,NaN,NaN,NaN,True,True,NaN,False,True,True,True,NaN,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,hhadrfosterprogram@outlook.com,262-968-3858,S47 W30760 State Road 59,NaN,North Prairie,WI,53153,US,58648075,dog,ms144,NaN,"The sturdy, well-balanced Labrador Retriever c...","Friendly, Active, Outgoing",1,54.61,62.23,24.947580,36.287390,10.0,12.0,Sporting Group,0.4,Weekly Brushing,0.8,Regularly,1.0,Needs Lots of Activity,1.0,Eager to Please,1.0,Outgoing
59894,58638621,TX1827,https://www.petfinder.com/dog/chick-58638621/t...,Dog,Dog,Young,Female,Medium,Medium,"['Friendly', 'Playful', 'Smart', 'Athletic']",Chick,Chick is a gorgeous high energy girl that need...,NaN,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,[],adoptable,2022-10-24T02:05:16+0000,2022-10-24T02:05:16+0000,NaN,Australian Kelpie,Border Collie,True,False,Bicolor,NaN,NaN,True,True,NaN,False,True,True,True,NaN,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,seachicken1954@yahoo.com,(979) 279-6388,NaN,NaN,Hearne,TX,77859,US,58638621,dog,tx1827,NaN,"As a herding dog, the Australian Kelpie is int...","Loyal, Alert, Intelligent",NaN,43.18,50.80,NaN,NaN,10.0,13.0,Foundation Stock Service,0.4,Weekly Brushing,0.8,Regularly,0.8,Energetic,1.0,Eager to Please,0.6,Alert/Responsive
77505,57641502,CA2489,https://www.petfinder.com/dog/bailey-57641502/...,Dog,Dog,Senior,Female,Medium,NaN,[],Bailey,Meet Bailey!\n\nPretty girl Bailey is the perf...,17149959,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,[],adoptable,2022-09-27T20:03:02+0000,2022-09-27T20:03:02+0000,NaN,American Staffordshire Terrier,NaN,True,False,NaN,NaN,NaN,True,True,NaN,False,True,True,True,True,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,doggiebonezrescue@gmail.com,(714) 381-4344,NaN,NaN,San Juan Capistrano,CA,92675,US,57641502,dog,ca2489,NaN,"AmStaffs are stocky, muscular bull-type terrie...","Confident, Smart, Good-Natured",85,43.18,48.26,18.143695,31.751466,12.0,16.0,Terrier Group,0.2,Occasional Bath/Brush,0.4,Occasional,0.6,Regular Exercise,0.6,Agreeable,0.6,Alert/Responsive


In [5]:
dogs_DF.columns

Index(['id', 'organization_id', 'url', 'type', 'species', 'age', 'gender',
       'size', 'coat', 'tags', 'name', 'description_x',
       'organization_animal_id', 'photos', 'videos', 'status',
       'status_changed_at', 'published_at', 'distance', 'breeds.primary',
       'breeds.secondary', 'breeds.mixed', 'breeds.unknown', 'colors.primary',
       'colors.secondary', 'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'primary_photo_cropped.small', 'primary_photo_cropped.medium',
       'primary_photo_cropped.large', 'primary_photo_cropped.full',
       'contact.email', 'contact.phone', 'contact.address.address1',
       'contact.address.address2', 'contact.address.city',
       'contact.address.state', 'contact.address.postcode',
       'contact.address.country', 'animal_id', 'animal_type

Drop animals with no pictures since they are key to our 'tinder-like' app experience.

In [6]:
dogs_DF = dogs_DF.dropna(subset=['primary_photo_cropped.full'])# drop rows with 0 pictures
dogs_DF.shape # matches na count via sweet viz for dogs

(97459, 70)

Next we seperate the dataframe into features to model over and context data that can be shown to the user for any matches. 'ID' will be our shared key between the two tables.

Of note, the 'distance' field and 'primary_photo_cropped.full' field will be useful data for future model enhancements. For the models so far, we will simply use textual data and assume a 0 distance for all pets.

In [7]:
contextCols = ['id','organization_id','url','type','tags','name','description_x','organization_animal_id',
              'photos','primary_photo_cropped','videos','status','status_changed_at','published_at',
              'distance','contact.email', 'contact.phone', 'contact.address.address1',
               'contact.address.address2', 'contact.address.city','contact.address.state', 
               'contact.address.postcode','contact.address.country', 'animal_id', 'animal_type',
               'organization_id.1', 'primary_photo_cropped.small','primary_photo_cropped.medium',
               'primary_photo_cropped.large','primary_photo_cropped.full','description_y',
               'temperament','popularity','min_height','max_height','min_weight','max_weight',
               'min_expectancy','max_expectancy','grooming_frequency_value','shedding_value',
               'energy_level_value','trainability_value','demeanor_value']
featureCols = ['id','age','gender','size','coat','breeds.primary', 'breeds.secondary','breeds.mixed',
              'breeds.unknown','colors.primary','colors.secondary','colors.tertiary',
              'attributes.spayed_neutered','attributes.house_trained','attributes.declawed',
              'attributes.special_needs','attributes.shots_current','environment.children',
              'environment.dogs','environment.cats','type','contact.address.postcode',
              'group','grooming_frequency_category','shedding_category','energy_level_category',
               'trainability_category','demeanor_category'] # initial columns to keep for training purposes
dogs_DF_features = dogs_DF[featureCols]
dogs_DF_context = dogs_DF[contextCols]
dogs_DF_features.shape

(97459, 28)

Let's sanity check our missing values now that we just have dogs and remove any columns with too many missing values.

In [8]:
valueCounts = dogs_DF_features.set_index('type').isna().groupby(level=0).sum()/dogs_DF_features.shape[0] # level=0 refers to our index, which we made 'type'


In [9]:
pd.set_option('display.max_columns', 500)
valueCounts 

,id,age,gender,size,coat,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,contact.address.postcode,group,grooming_frequency_category,shedding_category,energy_level_category,trainability_category,demeanor_category
type,,,,,,,,,,,,,,,,,,,,,,,,,,,
Dog,0.0,0.0,0.0,0.0,0.660739,0.0,0.632676,0.0,0.0,0.455494,0.659888,0.88548,0.0,0.0,1.0,0.0,0.0,0.667717,0.545347,0.801824,0.000195,0.519429,0.526652,0.526888,0.519439,0.52027,0.52027


In [10]:
valueCounts = dogs_DF_context.set_index('type').isna().groupby(level=0).sum()/dogs_DF_context.shape[0] # level=0 refers to our index, which we made 'type'


In [11]:
pd.set_option('display.max_columns', 500)
valueCounts 

,id,organization_id,url,tags,name,description_x,organization_animal_id,photos,primary_photo_cropped,videos,status,status_changed_at,published_at,distance,contact.email,contact.phone,contact.address.address1,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,animal_id,animal_type,organization_id.1,primary_photo_cropped.small,primary_photo_cropped.medium,primary_photo_cropped.large,primary_photo_cropped.full,description_y,temperament,popularity,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,grooming_frequency_value,shedding_value,energy_level_value,trainability_value,demeanor_value
type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Dog,0.0,0.0,0.0,0.0,0.0,0.228722,0.345089,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.062847,0.246052,0.439457,0.943053,0.000154,0.000154,0.000195,0.000154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.519429,0.519429,0.555659,0.519429,0.519429,0.522127,0.522127,0.521173,0.521173,0.526652,0.526888,0.519439,0.52027,0.52027


After a quick NA check, we will have to remove 'coat','breeds.secondary','colors.secondary','colors.tertiary', and 'attributes.declawed' (doesn't make sense for dogs). The column 'colors.primary' is also missing a lot of values but for sake of differing one dog from another it will be kept for now. Additionally, we will bring back in address postcode as an initial attempt to match nearby dogs together. Also, 'environment.children','environment.dogs', and 'environment.cats' have a lot of missing values but users derive a lot of value from this information. Lastly, the AKC data has a lot of missing data but that is only because the dataset has a lot of mixed breeds and AKC only handles purebreds. The value for purebreds is significant. Therefore, they will be kept as well.

In [12]:
featureCols = ['id','age','gender','size','breeds.primary','breeds.mixed',
               'colors.primary','attributes.spayed_neutered','attributes.house_trained',
               'attributes.special_needs','attributes.shots_current',
               'contact.address.postcode','environment.children','environment.dogs','environment.cats',
               'group','grooming_frequency_category','shedding_category',
               'energy_level_category', 'trainability_category', 'demeanor_category']

dogs_DF_features = dogs_DF[featureCols]
dogs_DF_context = dogs_DF[contextCols]
dogs_DF_features.shape

(97459, 21)

In [13]:
dogs_DF_features.dtypes

id                              int64
age                            object
gender                         object
size                           object
breeds.primary                 object
breeds.mixed                     bool
colors.primary                 object
attributes.spayed_neutered       bool
attributes.house_trained         bool
attributes.special_needs         bool
attributes.shots_current         bool
contact.address.postcode       object
environment.children           object
environment.dogs               object
environment.cats               object
group                          object
grooming_frequency_category    object
shedding_category              object
energy_level_category          object
trainability_category          object
demeanor_category              object
dtype: object

# Pre-process feature data<a id='pre-process'></a>

## Make Needed Helper Functions and Imports <a id='pp_pipeline'></a>

Make needed helper functions for modeling later on in workbook.

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.metrics.pairwise import laplacian_kernel
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [15]:
def remove_columns_with_1_distinct(df):
    drop_col = [e for e in df.columns if df[e].nunique()==1]
    df_return = df.drop(drop_col,axis=1)
    return df_return


In [16]:
def drop_duplicates(df):
    df_return = df.drop_duplicates()
    return df_return


In [17]:
def linear_similarities(df_1,id_df):
    cs_simil = linear_kernel(df_1,df_1)
    results = {}
    ds = id_df # needs id column
    for idx, row in ds.iterrows():
       similar_indices = cs_simil[idx].argsort()[:-100:-1] 
       similar_items = [(cs_simil[idx][i], ds['id'][i]) for i in similar_indices] 
       results[row['id']] = similar_items[1:]
    return results

In [18]:
def cosine_similarities(df_1,id_df):
    cs_simil = cosine_similarity(df_1,df_1)
    results = {}
    ds = id_df # needs id column
    for idx, row in ds.iterrows():
       similar_indices = cs_simil[idx].argsort()[:-100:-1] 
       similar_items = [(cs_simil[idx][i], ds['id'][i]) for i in similar_indices] 
       results[row['id']] = similar_items[1:]
    return results

In [19]:
def laplacian_similarities(df_1,id_df):
    cs_simil = laplacian_kernel(df_1,df_1)
    results = {}
    ds = id_df # needs id column
    for idx, row in ds.iterrows():
       similar_indices = cs_simil[idx].argsort()[:-100:-1] 
       similar_items = [(cs_simil[idx][i], ds['id'][i]) for i in similar_indices] 
       results[row['id']] = similar_items[1:]
    return results

In [20]:
def item(id,df):  
    ds = df
    colsGrab = ['id']
    return ds.loc[ds['id'] == id][colsGrab].values[0]# Just reads the results out of the dictionary.

def url(id,df):  
    ds = df
    colsGrab = ['url']
    return ds.loc[ds['id'] == id][colsGrab].values[0]# Just reads the results out of the dictionary.

def picture(id,df):  
    ds = df
    colsGrab = ['primary_photo_cropped.full']
    return ds.loc[ds['id'] == id][colsGrab].values[0]# Just reads the results out of the dictionary.

def recommend(item_id, num,df,reccs):
    print("Recommending " + str(num) + " dogs similar to " + str(item(item_id,df)) + "... " 
          + picture(item_id,df) + " - " + url(item_id,df))   
    print("-------")    
    recs = reccs[item_id][:num]   
    for rec in recs: 
        print("Recommended: " + str(item(rec[1],df)) + " (score:" +      str(rec[0]) + ") " 
              + picture(rec[1],df) + " - " + url(rec[1],df))
    
def score(reccs, num):
    print("Finding average reccomendation score for top 5 reccomendations per example")
    results = []
    for key in reccs.keys():
        subRecs = reccs[key][:num]
        for r in subRecs:
            results.append(r[0])
    averageRecc = sum(results) / len(results)
    print("There are "+ str(len(results)) + 'results with a sum of' + str(sum(results)) + 'and and average of: ' 
          + str(averageRecc) )
    return averageRecc

# Collaborative Filtering - Under Construction WIP <a id='cf'></a>

##Surprise Method

In [21]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366473 sha256=148843cfb3b594fe7c6f1dec205a9d00b104537574ed91b47379e8782e7c85f2
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [22]:
from surprise import Dataset, SVD, accuracy
from surprise.model_selection import cross_validate
from surprise import BaselineOnly, Reader
import pandas as pd
from sklearn.model_selection import train_test_split
# importing product
from itertools import product

In [23]:
dog_rankings = pd.read_csv("/content/drive/MyDrive/MLE10PetMatch/petmatch_rankings_dogs.csv",header=0)
dog_rankings.shape

(159, 3)

In [24]:
dog_rankings =dog_rankings.rename(columns={"user_name":"user_name","dog_id":"dog_id","preference":"raw_ratings"})
dog_rankings.head(3) # dataframe already in order we desire but renaming preferences

,user_name,dog_id,raw_ratings
0,Denise,58998198,1
1,Denise,58960846,0
2,Denise,57409728,0


In [25]:
dog_rankings.groupby('user_name').count() # get idea of rankings data so far

,dog_id,raw_ratings
user_name,,
2,45,45
Denise,46,46
Matt,18,18
Tibor,23,23
Zsofi,27,27


In [26]:
'''
rating dataframe will look like this
| user_id | item_id | rating          |
|---------|---------|-----------------|
| 1       | 1       | 5               |
| ...     | ...     | ...             |
| n       | m       | 3               |
'''

'\nrating dataframe will look like this\n| user_id | item_id | rating          |\n|---------|---------|-----------------|\n| 1       | 1       | 5               |\n| ...     | ...     | ...             |\n| n       | m       | 3               |\n'

In [27]:
reader=Reader(rating_scale=(0,1))
data= Dataset.load_from_df(dog_rankings,reader)

In [28]:
type(data)

surprise.dataset.DatasetAutoFolds

In [29]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True) # initial results on cat_rankings with no fillers and no datasets

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.3908  0.4692  0.4388  0.5233  0.4041  0.4205  0.3530  0.4751  0.4618  0.3772  0.4314  0.0494  
MAE (testset)     0.3415  0.4025  0.3809  0.4275  0.3319  0.3670  0.3208  0.3823  0.3811  0.3227  0.3658  0.0339  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.39075079, 0.46916501, 0.43882078, 0.52329249, 0.40409315,
        0.42047801, 0.35298987, 0.47506271, 0.46183528, 0.37716758]),
 'test_mae': array([0.34151866, 0.40251683, 0.38093701, 0.42750128, 0.33194474,
        0.36704404, 0.32083802, 0.38234473, 0.38106834, 0.32271023]),
 'fit_time': (0.0026235580444335938,
  0.0028104782104492188,
  0.0020139217376708984,
  0.0020210742950439453,
  0.0025708675384521484,
  0.002418994903564453,
  0.002707958221435547,
  0.002070188522338867,
  0.002022266387939453,
  0.0021381378173828125),
 'test_time': (0.0001709461212158203,
  0.00013971328735351562,
  0.00010132789611816406,
  0.0001285076141357422,
  0.00010204315185546875,
  0.0001087188720703125,
  0.00011587142944335938,
  9.870529174804688e-05,
  9.655952453613281e-05,
  9.489059448242188e-05)}

Inital CV results are not horrible, but was worst than cats. let's do the full real go at it

In [30]:
trainset= data.build_full_trainset()
svd.fit(trainset) # model fit

In [31]:
type(trainset)

surprise.trainset.Trainset

In [32]:
# Let's check Denise's ratings
dog_rankings[dog_rankings['user_name']=="Denise"]

,user_name,dog_id,raw_ratings
0,Denise,58998198,1
1,Denise,58960846,0
2,Denise,57409728,0
3,Denise,58806727,0
4,Denise,58733020,0
5,Denise,58977774,0
6,Denise,58816760,0
7,Denise,58865203,0
8,Denise,59012942,0
9,Denise,57698311,1


In [33]:
# Let's check someone else's ratings to find one Denise did not rate
dog_rankings[dog_rankings['user_name']=="Matt"]

,user_name,dog_id,raw_ratings
46,Matt,56856439,1
47,Matt,58918611,0
48,Matt,58987604,0
49,Matt,58933248,1
50,Matt,58880510,0
51,Matt,56757450,1
52,Matt,59010353,0
53,Matt,58075196,1
54,Matt,58294762,1
55,Matt,57333201,0


In [39]:
svd.predict("Denise",56544201,1)

Prediction(uid='Denise', iid=56544201, r_ui=1, est=0.4141225174781038, details={'was_impossible': False})

In [34]:
svd.predict("Denise",56856439) #version if haven't rated it yet

Prediction(uid='Denise', iid=56856439, r_ui=None, est=0.35671964716393656, details={'was_impossible': False})

In [41]:
# Try Collab Filtering with dog train data
train_df= dog_rankings
reader=Reader(rating_scale=(0,1))
train_data= Dataset.load_from_df(train_df,reader)

In [42]:
trainset_full= train_data.build_full_trainset() #REQUIRED
testset_full= trainset_full.build_testset()

In [43]:
cross_validate(SVD(), train_data, cv=2) # still not as good as cats..

{'test_rmse': array([0.44563908, 0.44606599]),
 'test_mae': array([0.37255709, 0.37232829]),
 'fit_time': (0.0028405189514160156, 0.0017974376678466797),
 'test_time': (0.0008273124694824219, 0.0005435943603515625)}

In [44]:
algo = SVD()
algo.fit(trainset_full)
predictions = algo.test(testset_full,verbose=True)
accuracy.rmse(predictions)  # slightly better RMSE than CV but not as good as cats


user: Denise     item: 58998198   r_ui = 1.00   est = 0.32   {'was_impossible': False}
user: Denise     item: 58960846   r_ui = 0.00   est = 0.11   {'was_impossible': False}
user: Denise     item: 57409728   r_ui = 0.00   est = 0.16   {'was_impossible': False}
user: Denise     item: 58806727   r_ui = 0.00   est = 0.16   {'was_impossible': False}
user: Denise     item: 58733020   r_ui = 0.00   est = 0.11   {'was_impossible': False}
user: Denise     item: 58977774   r_ui = 0.00   est = 0.28   {'was_impossible': False}
user: Denise     item: 58816760   r_ui = 0.00   est = 0.28   {'was_impossible': False}
user: Denise     item: 58865203   r_ui = 0.00   est = 0.18   {'was_impossible': False}
user: Denise     item: 59012942   r_ui = 0.00   est = 0.14   {'was_impossible': False}
user: Denise     item: 57698311   r_ui = 1.00   est = 0.50   {'was_impossible': False}
user: Denise     item: 58932466   r_ui = 0.00   est = 0.23   {'was_impossible': False}
user: Denise     item: 58575631   r_ui = 0.

0.3213848496309936

In [45]:
dog_rankings.head(3)

,user_name,dog_id,raw_ratings
0,Denise,58998198,1
1,Denise,58960846,0
2,Denise,57409728,0


In [46]:
joblib.dump(algo, '/content/drive/MyDrive/MLE10PetMatch/models/collabfilter_model_dogs_v1.pkl')

['/content/drive/MyDrive/MLE10PetMatch/models/collabfilter_model_dogs_v1.pkl']

In [47]:
algo_t = joblib.load('/content/drive/MyDrive/MLE10PetMatch/models/collabfilter_model_dogs_v1.pkl')

In [48]:
predictions = algo_t.test(testset_full,verbose=True)
accuracy.rmse(predictions) 

user: Denise     item: 58998198   r_ui = 1.00   est = 0.32   {'was_impossible': False}
user: Denise     item: 58960846   r_ui = 0.00   est = 0.11   {'was_impossible': False}
user: Denise     item: 57409728   r_ui = 0.00   est = 0.16   {'was_impossible': False}
user: Denise     item: 58806727   r_ui = 0.00   est = 0.16   {'was_impossible': False}
user: Denise     item: 58733020   r_ui = 0.00   est = 0.11   {'was_impossible': False}
user: Denise     item: 58977774   r_ui = 0.00   est = 0.28   {'was_impossible': False}
user: Denise     item: 58816760   r_ui = 0.00   est = 0.28   {'was_impossible': False}
user: Denise     item: 58865203   r_ui = 0.00   est = 0.18   {'was_impossible': False}
user: Denise     item: 59012942   r_ui = 0.00   est = 0.14   {'was_impossible': False}
user: Denise     item: 57698311   r_ui = 1.00   est = 0.50   {'was_impossible': False}
user: Denise     item: 58932466   r_ui = 0.00   est = 0.23   {'was_impossible': False}
user: Denise     item: 58575631   r_ui = 0.

0.3213848496309936

# Conclusion and Next Steps <a id='conclusion'></a>

**Conclusion of ML Modeling as of 1/18/23:**


*   Cosine Similarity with all examples improves the performance by .004
*   Will now work with UI and only slightly expanded dog set





**Conclusion of ML Modeling as of 1/2/23**: 
- All three content-based filtering models perform well
- Cosine Similarity appears to be the most sensitive to differences and has a very useful scale of 0-1.
- Can hook up content-based filtering models to PetMatch UI as-is and it should return good results based on overall similarity measures measured so far.
- User Rankings Data generated require more formating than initially expected but our application tracks all the key required fields for now.
- Collaborative Filtering is harder to implement than initially expected, but we have initial data to give it a try.

**Conclusion of ML Baseline as of 12/6/22**: 
- Average top 5 recommendation per cat in the training set is 10.96. The highest available score is a 12.  
- The result above uses a simple content-based filtering recommendation model without using user perferences, since they are currently not available. Instead it compares items against each other, aka you liked this ketchup so here are 10 other similar types of ketchup. 
- Due to the method used to create the simple content-based filtering model, dev and test set can not be used so to get an initial idea of the results the training set was used. 
- The cats data version 0.5 features need more ways to dileanate one cat from another but based on include visual scans and the average reccomendation score, the simple cat CBF model generally excels at giving you similar cats to what you stated you wanted.
- In instances where there is more ambiguity (aka a chosen cat with less defined details), it will still find cats very similar to it but sometimes it can also throw in very similar cats who are a different breed. This might not be a bad thing.

**Next Steps**:

- Get more user rankings!
- Incorporate distance more effectively
- Use cosine similarity with all dogs!
- Collaborative Filtering for item and user-based
  - Use surprise library possibly
  - Add timestamp to rankings so we can be time-sensitive in terms of reccomendations